# Assignment 4b - Graph Convolutional Networks
## Deep Learning Course - Vrije Universiteit Amsterdam, 2022

#### Instructions on how to use this notebook:

This notebook is hosted on Google Colab. To be able to work on it, you have to create your own copy. Go to *File* and select *Save a copy in Drive*.

You can also avoid using Colab entirely, and download the notebook to run it on your own machine. If you choose this, go to *File* and select *Download .ipynb*.

The advantage of using Colab is that you can use a GPU. You can complete this assignment with a CPU, but it will take a bit longer. Furthermore, we encourage you to train using the GPU not only for faster training, but also to get experience with this setting. This includes moving models and tensors to the GPU and back. This experience is very valuable because for many interesting models and large datasets (like large CNNs for ImageNet, or Transformer models trained on Wikipedia), training on GPU is the only feasible way.

The default Colab runtime does not have a GPU. To change this, go to *Runtime - Change runtime type*, and select *GPU* as the hardware accelerator. The GPU that you get changes according to what resources are available at the time, and its memory can go from a 5GB, to around 18GB if you are lucky. If you are curious, you can run the following in a code cell to check:

```sh
!nvidia-smi
```

Note that despite the name, Google Colab does  not support collaborative work without issues. When two or more people edit the notebook concurrently, only one version will be saved. You can choose to do group programming with one person sharing the screen with the others, or make multiple copies of the notebook to work concurrently.

**Submission:** Upload your notebook in .ipynb format to Canvas. The code and answers to the questions in the notebook are sufficient, no separate report is expected. 

In [1]:
!nvidia-smi

Fri Dec 23 23:17:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.49       Driver Version: 457.49       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   38C    P8     4W /  N/A |    134MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Introduction

Graphs are very useful data structures that allow us to represent sets of entities and the way they are related among each other. In a graph, entities are also known as *nodes*, and any link between entities is also called an *edge*.

Examples of real world objects that can be modeled as graphs are social networks, where entities are people and relations denote friendship; and molecules, where entities are atoms and relations indicate a bond between them.

There has been increased interest in the recent years in the application of deep learning architectures to graph-structured data, for tasks like predicting missing relations between entities, classifying entities, and classifying graphs. This interest has been spurred by the introduction of Graph Convolutional Networks (GCNs).

In this assignment, you will implement and experiment with one of the first versions of the GCN, proposed by Thomas Kipf and Max Welling in their 2017 paper, [Semi-supervised Classification with Graph Convolutional Networks](https://arxiv.org/abs/1609.02907). In particular, the goals of this assignment are to

- Understand how GCNs are formulated
- Implement the GCN using PyTorch
- Train and evaluate a model for semi-supervised node classification in citation networks
- Train and evaluate a model for binary classification of molecules

### Representing graphs

Suppose we have the following graph:

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/01-graph.png" width="200">

This is an undirected graph (since the edges have no specified direction) with 4 nodes. One way to represent the connectivity structure of the graph is by means of the **adjacency matrix**. The $i$-th row of the matrix contains a 1 in the $j$-th column, if nodes $i$ and $j$ are connected. For an undirected graph like the one above, this means that the adjacency matrix

- Is symmetric (e.g. an edge between 0 and 2 is equivalent as an edge between 2 and 0)
- Is square, of size $n\times n$ where $n$ is the number of nodes

The adjacency matrix for the graph above is then the following:

$$
A =
\begin{bmatrix}
0 & 0 & 1 & 0 \\ 
0 & 0 & 1 & 0 \\
1 & 1 & 0 & 1 \\
0 & 0 & 1 & 0
\end{bmatrix}
$$

A second matrix of interest is the **degree matrix**. This is a diagonal matrix where the $i$-th element of the diagonal indicates the number of edges connected to node $i$. Note that these can be obtained from $A$ by summing across the columns, or the rows. For our example, the degree matrix is

$$
D = \begin{bmatrix}
1 & 0 & 0 & 0 \\ 
0 & 1 & 0 & 0 \\
0 & 0 & 3 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

For specific applications, each node in the graph will have an associated vector of features $x\in\mathbb{R}^c$. If our graph is a social network, then the vector of features can contain information like age, location, and musical tastes, in a specific numeric format. In the case of a molecule, the node could represent an atom and have features like the atomic mass, etc. We can lay out the features in a matrix $X\in\mathbb{R}^{n\times c}$, so that the feature vector for node $i$ is in the $i$-th row.

### Loading a citation network

To move to a real world example, we will start with the Cora dataset. This dataset represents a citation network, where nodes are scientific publications, edges denote citations between them, and features are a [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) extracted from their contents.

This graph contains labels for nodes, that represent a specific topic. We will use these for a node classification task.

To easily load it, we will use [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/index.html) (PyG), a deep learning library for graph-structured data.

In [2]:
# # Install PyTorch Geometric
# import torch
# try:
#     import torch_geometric
# except ModuleNotFoundError:
#     # Installing torch geometric packages with specific CUDA+PyTorch version.
#     # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details
#     TORCH = torch.__version__.split('+')[0]
#     CUDA = 'cu' + torch.version.cuda.replace('.','')

#     !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#     !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#     !pip install torch-geometric
#     import torch_geometric

We can now use the library to download and import the dataset. Initializing the `Planetoid` class returns a `Dataset` object that can contain multiple graphs. In this task we will only use the `Cora` dataset (the citation network) and hence, we will select only the first element.

In [3]:
from torch_geometric.datasets import Planetoid
data = Planetoid(root='data/Planetoid', name='Cora')[0]


#### Question 1 (0.25 pt)

The `data` object is an instance of the `Data` class in PyG. Check the [documentation](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html) and report the following properties of the graph:

- Number of nodes
- Number of edges 
- The dimension $c$ of the feature vectors $x\in\mathbb{R}^c$
- The number of targets for the classification task

In [4]:
print('Number of nodes: ', data.num_nodes)
print('Number of edges: ', data.num_edges)
print(f'Number of features: {data.num_node_features}')
dataset = Planetoid(root='data/Planetoid', name='Cora')
print(f'Number of classes: {dataset.num_classes}')

Number of nodes:  2708
Number of edges:  10556
Number of features: 1433
Number of classes: 7


#### Question 2 (0.25 pt)

In PyG, edges are provided in a tensor of shape (2, number of edges). You can access it via `data.edge_index`. Each column in this tensor contains the IDs for two nodes that are connected in the graph.

We saw that in an undirected graph, an edge between nodes $i$ and $j$ adds a value of 1 to positions $(i, j)$ and $(j, i)$ of the adjacency matrix. Is this also true for the edge index? That is, if there is an edge $(i, j)$ in `data.edge_index`, is there also an edge for $(j, i)$? This is important to know for the next steps of the implementation.

In [5]:
import numpy as np

print(data.edge_index)
a=data.edge_index
# The last pair (i,j) is (2707,2706), so (j,i) should be pretty close from the RHS of data.edge_index.

# Thus, we can check whether (j,i)=(2706,2707) appears in data.edge_index easily:
print(a[0][-5:])
print(a[1][-5:])

# The answer is Yes! the edge index contains both (i,j) an (j,i) edges.

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])
tensor([2706, 2707, 2707, 2707, 2707])
tensor([2707,  165,  598, 1473, 2706])


#### Question 3 (0.5 pt)

In graphs, especially large ones, the adjacency matrix is **sparse**: most entries are zero. Sparse matrices allow for efficient storage and computation.

To prepare and pre-process sparse matrices, we will use [`scipy.sparse`](https://docs.scipy.org/doc/scipy/reference/sparse.html). Once the matrices are ready, we will convert them to PyTorch tensors.

We will use the [Sparse COO format](https://en.wikipedia.org/wiki/Sparse_matrix#Coordinate_list_(COO)). We encourage you to first get familiar with how it works after continuing with the assignment.

- Use the [`scipy.sparse.coo_matrix()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) function to build the adjacency matrix. Think of what arguments are needed, and how you can obtain them from the graph data loaded above.
- Use the `sum()` method of sparse matrices, together with `scipy.sparse.diags()`, to compute the degree matrix using the definition above.

Both resulting matrices must be sparse of type `float32`.

In [6]:
from torch_sparse.tensor import scipy

# Your answer here
from torch_geometric.datasets import Planetoid
data = Planetoid(root='data/Planetoid', name='Cora')[0]

import numpy as np

# Get the number of nodes in the graph
num_nodes = data.num_nodes

# Build the adjacency matrix
adjacency_matrix = scipy.sparse.coo_matrix((np.ones(data.edge_index.shape[1]),
                  (data.edge_index[0, :], data.edge_index[1, :])),
                  shape=(num_nodes, num_nodes), dtype=np.float32)
print("Adjacency matrix")
print(adjacency_matrix)

# Get the degree vector
degree_vector = adjacency_matrix.sum(axis=1).A1

# Build the degree matrix
degree_matrix = scipy.sparse.diags([degree_vector], [0], dtype=np.float32)
print("\nDegree matrix")
print(degree_matrix)


Adjacency matrix
  (0, 633)	1.0
  (0, 1862)	1.0
  (0, 2582)	1.0
  (1, 2)	1.0
  (1, 652)	1.0
  (1, 654)	1.0
  (2, 1)	1.0
  (2, 332)	1.0
  (2, 1454)	1.0
  (2, 1666)	1.0
  (2, 1986)	1.0
  (3, 2544)	1.0
  (4, 1016)	1.0
  (4, 1256)	1.0
  (4, 1761)	1.0
  (4, 2175)	1.0
  (4, 2176)	1.0
  (5, 1629)	1.0
  (5, 1659)	1.0
  (5, 2546)	1.0
  (6, 373)	1.0
  (6, 1042)	1.0
  (6, 1416)	1.0
  (6, 1602)	1.0
  (7, 208)	1.0
  :	:
  (2694, 431)	1.0
  (2694, 2695)	1.0
  (2695, 431)	1.0
  (2695, 2694)	1.0
  (2696, 2615)	1.0
  (2697, 986)	1.0
  (2698, 1400)	1.0
  (2698, 1573)	1.0
  (2699, 2630)	1.0
  (2700, 1151)	1.0
  (2701, 44)	1.0
  (2701, 2624)	1.0
  (2702, 186)	1.0
  (2702, 1536)	1.0
  (2703, 1298)	1.0
  (2704, 641)	1.0
  (2705, 287)	1.0
  (2706, 165)	1.0
  (2706, 169)	1.0
  (2706, 1473)	1.0
  (2706, 2707)	1.0
  (2707, 165)	1.0
  (2707, 598)	1.0
  (2707, 1473)	1.0
  (2707, 2706)	1.0

Degree matrix
  (0, 0)	3.0
  (1, 1)	3.0
  (2, 2)	5.0
  (3, 3)	1.0
  (4, 4)	5.0
  (5, 5)	3.0
  (6, 6)	4.0
  (7, 7)	1.0
  (8, 8

You might wonder why we suggest to use a scipy sparse matrix, while also PyTorch supports them. The reason is that in the next step, we will be multiplying two sparse matrices, an operation not supported in PyTorch. PyTorch only allows multiplying a sparse matrix with a dense one, something which we will be doing at a later stage.



### The Graph Convolutional Network

The goal of the graph convolution is to take the feature vectors of all nodes $X\in\mathbb{R}^{n\times c}$, and propagate them along the existing edges, to obtain updated representations $Z\in\mathbb{R}^{n\times d}$.


The GCN is initially motivated as performing a convolution, similarly as it is done in CNNs for images, for graph-structured data. In Kipf and Welling (2017), a theoretical derivation leads to the following formula:

$$
Z = \tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}XW
$$

where

- $W\in\mathbb{R}^{c\times d}$ is a matrix of parameters to be learned via gradient descent
- $\tilde{A} = A + I_n$, where $I_n$ is an $n\times n$ identity matrix
- $\tilde{D}$ is the degree matrix computed with $\tilde{A}$ as the adjacency matrix

If we define $\hat{A} = \tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}$, the graph convolution can be written as $Z = \hat{A}XW$.

#### Question 4 (0.25 pt)

Given the formula for the GCN, explain why it operates by propagating feature vectors across the graph. To answer this, it might be useful to recall the definitions of the adjacency and degree matrices, and how they are involved in the formula.

**Answer**

<p align="justify">The graph convolution operation in a GCN operates by propagating feature vectors across the graph because it uses the adjacency matrix, which encodes the connections between nodes in the graph, to weight the contribution of each node's feature vector to the updated representations of its neighbors.

<p align="justify">The adjacency matrix $A\in\mathbb{R}^{n\times n}$ is a square matrix where n is the number of nodes in the graph. Each element $A_{ij}$ in the matrix represents the strength of the connection between nodes i and j. If there is no connection between the two nodes, the value of $A_{ij}$ is 0. If there is a connection, the value is 1 (for an weighted graph it can also be a real number).

<p align="justify">The degree matrix $D\in\mathbb{R}^{n\times n}$ is a diagonal matrix where the value of each element $D_{ii}$ is the degree of the node $i$, i.e., the number of connections it has to other nodes.

<p align="justify">In the GCN formula, the adjacency matrix is first transformed into $\tilde{A}$ by adding the identity matrix $I_n$ to it. This has the effect of adding self-loops to all the nodes, i.e., connections from each node to itself with weight 1. This is done to ensure that each node's own feature vector is taken into account when updating its representation.

<p align="justify">The degree matrix $\tilde{D}$ is then computed from the modified adjacency matrix $\tilde{A}$. The matrix $\tilde{A}$ is then element-wise multiplied by the inverse of $\tilde{D}$, resulting in the matrix $\hat{A}$. This matrix is used to weight the contribution of each node's feature vector to the updated representation of its neighbors.

<p align="justify">Finally, the feature vectors $X$ are multiplied by the matrix of parameters $W$ to obtain the updated representations $Z$. The resulting matrix $Z$ contains the updated feature vectors for all the nodes in the graph.

<p align="justify">In this way, the GCN formula propagates the feature vectors of the nodes across the graph, using the adjacency matrix to weight the contribution of each node's feature vector to its neighbors' updated representations.

#### Question 5 (0.5 pt)

Compute the **normalized adjacency matrix** $\hat{A}$. The result should be a sparse matrix.

In [7]:
from torch_sparse.tensor import scipy
from scipy.sparse.linalg import inv
from torch_geometric.datasets import Planetoid
data = Planetoid(root='data/Planetoid', name='Cora')[0]

import numpy as np

# Get the number of nodes in the graph
num_nodes = data.num_nodes

# Build the adjacency matrix
adjacency_matrix = scipy.sparse.coo_matrix((np.ones(data.edge_index.shape[1]),
                  (data.edge_index[0, :], data.edge_index[1, :])),
                  shape=(num_nodes, num_nodes), dtype=np.float32)


# Add the identity matrix to the adjacency matrix A
A_hat = adjacency_matrix + scipy.sparse.eye(num_nodes)
degree_A_hat = A_hat.sum(axis=0).A1

D_hat = scipy.sparse.diags([degree_A_hat], [0], dtype=np.float32)

# Compute the inverse square root of the degree array
degree_sqrt_inv = 1 / np.sqrt(degree_A_hat)

# Construct the matrix D^(-1/2) as a sparse diagonal matrix
D_sqrt_inv = scipy.sparse.diags([degree_sqrt_inv], [0], dtype=np.float32)

# normalize
A_hat_norm = D_sqrt_inv.dot(A_hat).dot(D_sqrt_inv)
print(A_hat_norm)

  (0, 0)	0.25
  (0, 633)	0.25
  (0, 1862)	0.22360679507255554
  (0, 2582)	0.25
  (1, 1)	0.25
  (1, 2)	0.20412415266036987
  (1, 652)	0.28867512941360474
  (1, 654)	0.3535533845424652
  (2, 1)	0.20412415266036987
  (2, 2)	0.16666667879725594
  (2, 332)	0.16666667879725594
  (2, 1454)	0.2886751401597465
  (2, 1666)	0.15430335304673193
  (2, 1986)	0.05025189181493417
  (3, 3)	0.4999999828857291
  (3, 2544)	0.4999999828857291
  (4, 4)	0.16666667879725594
  (4, 1016)	0.16666667879725594
  (4, 1256)	0.13608277249582912
  (4, 1761)	0.14433757007987325
  (4, 2175)	0.16666667879725594
  (4, 2176)	0.13608277249582912
  (5, 5)	0.25
  (5, 1629)	0.25
  (5, 1659)	0.28867512941360474
  :	:
  (2699, 2699)	0.4999999828857291
  (2700, 1151)	0.4082482761496564
  (2700, 2700)	0.4999999828857291
  (2701, 44)	0.28867512941360474
  (2701, 2624)	0.33333332136784577
  (2701, 2701)	0.33333332136784577
  (2702, 186)	0.21821788274037246
  (2702, 1536)	0.25819888202132546
  (2702, 2702)	0.33333332136784577
  (2703

#### Question 6 (0.5 pt)

So far we have used scipy to build and compute sparse matrices. Since we want to train a GCN with PyTorch, we need to convert $\hat{A}$ into a sparse PyTorch tensor. You can do this with the [`torch.sparse_coo_tensor()`](https://pytorch.org/docs/stable/generated/torch.sparse_coo_tensor.html) function, making sure to specify `torch.float` as the type.

In [8]:
import torch
from torch_sparse.tensor import scipy
from scipy.sparse.linalg import inv
from torch_geometric.datasets import Planetoid
data = Planetoid(root='data/Planetoid', name='Cora')[0]

import numpy as np

# Get the non-zero element indices and values of A_hat_norm
row_indices, col_indices, values = scipy.sparse.find(A_hat_norm)

# Convert A_hat_norm to a PyTorch sparse tensor
A_hat_norm_torch = torch.sparse_coo_tensor(torch.LongTensor([row_indices, col_indices]),
                                           torch.FloatTensor(values),
                                           A_hat_norm.shape)

print(A_hat_norm_torch)
# Notice the warning. In this specific case, the warning message can be safely ignored because the list passed 
# to the LongTensor and FloatTensor constructors only have two elements, so the overhead of creating the list 
# is negligible. However, if we were to create a PyTorch tensor from a large list of NumPy arrays, we should 
# to consider following the suggestion in the warning message to improve performance.

tensor(indices=tensor([[   0,  633, 1862,  ..., 1473, 2706, 2707],
                       [   0,    0,    0,  ..., 2707, 2707, 2707]]),
       values=tensor([0.2500, 0.2500, 0.2236,  ..., 0.2000, 0.2000, 0.2000]),
       size=(2708, 2708), nnz=13264, layout=torch.sparse_coo)


C:\Users\hh\AppData\Local\Temp/ipykernel_22412/3965892582.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  A_hat_norm_torch = torch.sparse_coo_tensor(torch.LongTensor([row_indices, col_indices]),


#### Question 7 (0.5 pt)

We now have all the ingredients to build a GCN layer. Implement a class (inheriting from `torch.nn.Module`) with a learnable matrix of weights $W\in\mathbb{R}^{c\times d}$. Make sure to

- Call this class `GCNLayer`
- The `__init__()` constructor should take as argument the number of input and output features.
- Use `torch.nn.init.kaiming_uniform_` to initialize $W$.
- Define the `forward` method, which takes as input $X$ and $\hat{A}$ and returns $Z$. Note that multiplications involving the sparse matrix $\hat{A}$ have to be done with `torch.spmm`. 

Once you have implemented the class, instantiate a layer with the correct number of input features for the Cora dataset, and a number of output features of your choice. Do a forward pass and report the shape of the output tensor.

In [9]:
import os
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
import torch
from torch import Tensor
import torch_geometric
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import Planetoid
import networkx as nx
from networkx.algorithms import community

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import math
data = Planetoid(root='data/Planetoid', name='Cora')[0]
dataset = Planetoid(root='data/Planetoid', name='Cora')

class GCNLayer(nn.Module):


    def __init__(self, in_features, out_features, bias=True):
        super(GCNLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight)
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output


# Get the number of input features for the Cora dataset
in_feats = data.num_features

# Choose the number of output features
out_feats = 7

# Instantiate the GCN layer
gcn_layer = GCNLayer(in_feats, out_feats)
print(gcn_layer)

inputs = data.x
adjs = A_hat_norm_torch

# Perform a forward pass through the GCN layer
output = gcn_layer.forward(inputs, A_hat_norm_torch)

# Print the shape of the output tensor
print(output.shape)

GCNLayer (1433 -> 7)
torch.Size([2708, 7])


#### Question 8 (0.5 pt)

As we have seen so far, the GCN layer implements a special type of linear transformation of the inputs. However, it is often beneficial in deep learning to stack multiple, non-linear transformations of the input features. Implement a second module class for a model with two GCN layers (use the module you implemented in the previous question).

- Call this class `GCN`
- The constructor must now take as input the number of input features, the output dimension of the first layer (this is the hidden layer), and the output dimension of the output layer.
- In the forward pass, add a ReLU activation function after the first layer.

In [10]:
import torch.nn as nn
import torch.nn.functional as F


class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GCNLayer(nfeat, nhid)
        self.gc2 = GCNLayer(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

### GCNs for semi-supervised node classification

Now that we have a GCN with two layers, we can test its performance in a node classification task. We will pass the input node features $X$ through the GCN layers, and the output will be of size $n\times k$ where $k$ is the number of classes (which you found in question 1). The label denotes the topic an article in the citation network belongs to (e.g. physics, computer science, etc.).

#### Question 9 (1.5 pt)

Note that the `data` object contains all labels (for all splits) in `data.y`, and binary masks for the train, validation, and test splits in `data.train_mask`, `data.val_mask`, and `data.test_mask`, respectively. These masks are the same size as `data.y`, and indicate which labels belong to which split.

- Create a GCN with two layers (using the class from the previous question), with 32 as the hidden dimension, and the number of output features equal to the number of classes in the Cora dataset.

- Use the Adam optimizer with a learning rate of 0.01.

- Implement a training loop for the GCN. At each step, pass $X$ and $\hat{A}$ to the GCN to obtain the logits. Compute the mean cross-entropy loss **only for the training instances**, using the binary masks.

- After each training step, evaluate the accuracy for the validation instances.

- Train for 100 epochs. Once training is finished, plot the training loss and validation accuracy (in a graph in function of the epoch number), and report the accuracy in the test set.

You should obtain an accuracy over 75% on both the validation and test sets. You can also compare your results with the original paper, which also contains results for the Cora dataset. Give a brief discussion on the results of your experiments.

Note that in contrast with other tasks, like image classification on some datasets, we don't use mini-batches here. The whole matrix of features and the adjacency is passed to the GCN in one step.

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
import os
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
import torch
from torch import Tensor
import torch_geometric
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import Planetoid
import networkx as nx
from networkx.algorithms import community

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
data = Planetoid(root='data/Planetoid', name='Cora')[0]

# Load the graph data
#data = Planetoid(root='data/Planetoid', name='Cora')[0]

import matplotlib.pyplot as plt

class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats, dropout=0.1, concat=True, bias=True):
        super(GCNLayer, self).__init__()
        self.in_feats = in_feats
        self.out_feats = out_feats
        self.W = nn.Parameter(torch.FloatTensor(in_feats, out_feats))
        self.use_bias = bias
        if self.use_bias:
            self.B = nn.Parameter(torch.FloatTensor(out_feats))
        self.reset_parameters()
        self.dropout = dropout
        self.concat = concat
        
    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.W)
        if self.use_bias:
            nn.init.zeros_(self.B)
    
    def forward(self, inputs, adjs):
        support = torch.mm(inputs, self.W)
        output = torch.spmm(adjs, support)
        if self.use_bias:
            return output + self.B


# Get the number of input features and number of classes for the Cora dataset
nfeat = data.num_features
nclass = dataset.num_classes

# Set the hidden dimension of the GCN
nhid = 32

# Create the GCN
gcn = GCN(nfeat, nhid, nclass)

# Use the Adam optimizer with a learning rate of 0.01
optimizer = torch.optim.Adam(gcn.parameters(), lr=0.01)

# Keep track of the training and validation accuracies
train_losses = []
val_accuracies = []

# Train the model for 100 epochs
for epoch in range(100):
    # Zero the gradients of the optimizer
    optimizer.zero_grad()
    
    # Get the input features and adjacency matrix
    X = data.x
    adj = data.edge_index
    
    # Pass the input features and adjacency matrix to the GCN to get the logits
    logits = gcn.forward(X, adj)
    
    # Get the labels for the training instances
    labels = data.y[data.train_mask]
    
    # Compute the mean cross-entropy loss for the training instances
    loss = F.nll_loss(logits[data.train_mask], labels)
    
    # Backpropagate the loss and take an optimization step
    loss.backward()
    optimizer.step()
    
    # Append the training loss for this epoch to the list
    train_losses.append(loss.item())

    preds = logits[data.val_mask].max(1)[1]

    # Compute the accuracy for the validation instances
    accuracy = preds.eq(labels).sum().item() / labels.size(0)

    # Append the validation accuracy for this epoch to the list
    val_accuracies.append(accuracy)


# # Plot the training and validation accuracies
# # plt.plot(train_accuracies, label='train')
# # plt.plot(val_accuracies, label='val')
# # plt.xlabel('Epoch')
# # plt.ylabel('Accuracy')
# # plt.legend()
# # plt.show()

# preds = logits[data.test_mask].max(1)[1]
# accuracy = preds.eq(labels).sum().item() / labels.size(0)
# print('Test accuracy:', accuracy)

# # Compute the test accuracy
# # test_accuracy = (logits[data.test_mask].argmax(dim=1) == y[data.test_mask]).float().mean().item()
# # print(f'Test accuracy: {test_accuracy:.4f}')

AttributeError: ignored

#### Question 10 (0.5 pt)

The paper introduces GCNs as a way to solve a *semi-supervised* classification problem.

- What makes this problem semi-supervised?
- What is the proportion of labeled data used for training with respect to labeled data in the validation and test sets? What is difference in this context with other benchmark tasks in machine learning, like image classification with MNIST?
- Why do you think the GCN performs well in this semi-supervised scenario?

*Answer*

<p align="justify">In the "Semi-supervised Classification with Graph Convolutional Networks" paper, the authors use a number of citation network datasets to evaluate their GCN model. These datasets consist of nodes representing papers and edges representing citations between papers. The nodes are labeled with the subject area of the paper, and the task is to predict the subject area of each node based on the labels of the other nodes and the structure of the citation network.


* *What makes this problem semi-supervised?*

<p align="justify">The problem is considered semi-supervised because only a portion of the nodes in the dataset are labeled, while the rest of the nodes are unlabeled. The GCN model is trained to predict the labels of the unlabeled nodes based on the labels of the labeled nodes and the structure of the citation network. This is in contrast to a supervised learning problem, in which all the training data is labeled, or an unsupervised learning problem, in which all the data is unlabeled.

* *What is the proportion of labeled data used for training with respect to labeled data in the validation and test sets?*

<p align="justify">In the experiments reported in the paper, the authors use a fixed amount of labeled data for training and evaluate the model on a fixed amount of labeled data in the validation and test sets. The proportion of labeled data used for training varies across the different datasets, but in general it is relatively small compared to the total amount of data. For example, in the Cora dataset, the authors use 140 labeled data points for training, 500 labeled data points for validation, and 1,000 labeled data points for testing. This is a small fraction of the total number of data points in the dataset, which is around 2,500.

* *What is difference in this context with other benchmark tasks in machine learning, like image classification with MNIST?*

<p align="justify">The use of a relatively small amount of labeled data for training is common in semi-supervised learning, as it is often the case that only a small amount of labeled data is available. In contrast, in tasks like image classification with MNIST, a much larger proportion of the data is typically used for training. For example, the MNIST dataset contains 60,000 training images and 10,000 test images, so the proportion of labeled data used for training is much higher.

* *Why do you think the GCN performs well in this semi-supervised scenario?*

<p align="justify">There are several reasons why the GCN model introduced in the "Semi-supervised Classification with Graph Convolutional Networks" paper performs well in a semi-supervised classification scenario.

<p align="justify">First, the GCN model is specifically designed to make use of both labeled and unlabeled data in a graph structure. It uses the graph structure and the labels of the labeled nodes to propagate information and make predictions for the unlabeled nodes. This allows it to make use of additional information about the relationships between the data points, which can improve the model's performance.

<p align="justify">Second, the GCN model uses a convolutional neural network (CNN) architecture, which has been shown to be effective for many types of data, including images and text. The GCN model extends the CNN architecture to graph-structured data by using graph convolutional layers, which can effectively capture local patterns and dependencies in the graph structure.

<p align="justify">Finally, the GCN model is trained using a variant of the cross-entropy loss function, which is a common choice for classification tasks. This loss function allows the model to learn to predict the correct class labels for the data points based on the input features and the labels of the labeled data points.

<p align="justify">Overall, the combination of these factors makes the GCN model well-suited to solving semi-supervised classification problems on graph-structured data.

### Loading a dataset of proteins

In the previous sections you learned how to pass the adjacency matrix of a graph with a couple of thousand of nodes, to classify each node with a particular label. A different and useful application of GCNs is graph classification.

In contrast with the previous part, where there was a single, big graph, in graph classification we have multiple graphs, and each graph can be assigned a label. In this part of the assignment you will implement a classifier for proteins.

[Proteins](https://en.wikipedia.org/wiki/Protein_(nutrient)) are parts of the buildings block of life. They consist of chains of amino acids, and can take many shapes. In the PROTEINS dataset, proteins are represented as graphs, where the nodes are amino acids, and an edge between them indicates that they are 6 [Angstroms](https://en.wikipedia.org/wiki/Angstrom) apart. All graphs have a binary label, where 1 means that the protein is not an enzyme.

We will start by loading and examining this dataset.

In [10]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='data/TU', name='PROTEINS', use_node_attr=True)

Extracting data/TU/PROTEINS/PROTEINS.zip
Processing...
Done!


#### Question 11 (0.25 pt)

Unlike in the previous part, where we selected the first element returned by the loading function, note that here we get all the elements returned by `TUDataset()`. `dataset` is an interable object, that has some similar behaviors as a Python list: you can call `len()` on it, and you can takes slices from it.

Each element in `dataset` is a `Data` object containing a graph that represents a protein. This is the same type of object that we used in the previous part to store the Cora citation network.

Knowing this, answer the following:

- How many proteins (graphs) are there in `dataset`?
- Take any protein from `dataset`. How many nodes and edges does it contain? What is its label? How many features does each node have?

In [15]:
# How many proteins (graphs) are there in dataset?
num_proteins = len(dataset)
print(f'There are {num_proteins} proteins in the dataset.')

# Take any protein from dataset. For instance, the first one
protein = dataset[0]

# How many nodes and edges does it contain? 
num_nodes = protein.num_nodes
num_edges = protein.num_edges
print(f'The protein contains {num_nodes} nodes and {num_edges} edges.')

# What is its label? 
label = protein.y[0]
print(f'The label for this protein is {label}.')

# How many features does each node have?
num_features = protein.num_features
print(f'Each node in this protein has {num_features} features.')

There are 1113 proteins in the dataset.
The protein contains 42 nodes and 162 edges.
The label for this protein is 0.
Each node in this protein has 4 features.


#### Question 12 (0.5 pt)

To properly train and evaluate our model, we need training, validation, and test splits.

For reproducibility purposes, we generate a random tensor of indices for you. Use it to extract the three splits from `dataset`.

For training, take 80% of the indices (starting from the first element in `indices`), then the following 10% for validation, and the remaining 10% for testing. You can use the indices to index `dataset`.

Call the resulting splits `train_dataset`, `valid_dataset`, and `test_dataset`.

In [11]:
# Don't erase the following three lines
import torch
torch.random.manual_seed(0)
indices = torch.randperm(len(dataset))

# Your answer here
train_size = int(len(indices) * 0.8)
valid_size = int(len(indices) * 0.1)
test_size = len(indices) - train_size - valid_size

train_dataset = [dataset[i] for i in indices[: train_size]]
valid_dataset = [dataset[i] for i in indices[train_size: train_size + valid_size]]
test_dataset  = [dataset[i] for i in indices[train_size + valid_size: ]]

print(f"Number of proteins in the training set: {len(train_dataset)}")
print(f"Number of proteins in the validation set: {len(valid_dataset)}")
print(f"Number of proteins in the test set: {len(test_dataset)}")

Number of proteins in the training set: 890
Number of proteins in the validation set: 111
Number of proteins in the test set: 112


### Working with a batch of graphs

When working with the Cora dataset, you used the information in `data.edge_index` to build the sparse normalized adjacency matrix $\hat{A}$ that is required by the GCN. We could do something similar here: for each graph, we build $\hat{A}$, and pass it to the GCN. However, if the number of graphs is big, this can really slow down training.

To avoid this, we will resort to a very useful trick that also allows us to reuse the same GCN you implemented previously. The trick makes it possible to do a forward pass through the GCN for multiple, disconnected graphs at the same time (instead of only one), much like when you train with mini-batches for other kinds of data.

Let's first revisit the propagation rule of the GCN, $Z = \hat{A}XW$, with an illustration (we have omitted the cells of $X$ and $W$ for clarity):

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/02-gcn-forward.png">

If we have multiple graphs, we can still use the same propagation rule, if we

- Set $\hat{A}$ to be a block diagonal matrix, where the blocks are the different adjacency matrices of the graphs
- Concatenate the feature matrices along the first dimension

This is illustrated in the following figure, for a batch of 3 graphs. Note that the elements outside of the blocks are zero.

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/02-gcn-batch-forward.png">

The resulting adjacency matrix $\hat{A}_B$ can also be built as a sparse matrix, and once we have it together with the concatenated matrix of features, the computation of the graph convolution is exactly the same as before. Note how this trick also allows us to process graphs with different sizes and structures.


#### Question 13 (0.5 pt)


Just as the citation network, the graphs in each of the datasets you created in Question 12 also have an `edge_index` attribute, which can be used to compute the normalized adjacency matrix $\hat{A}$, for each graph.

Reusing your code for Questions 3 and 5, define a function `get_a_norm()` that takes as input an element of a dataset (e.g. `train_dataset[0]`), and returns a `scipy.sparse` matrix containing $\hat{A}$.

Note that an element of a dataset has properties like `num_edges`, `num_nodes`, etc. which you can use here.

In [14]:
def get_a_norm(data):
    num_nodes = data.num_nodes

    # Build the adjacency matrix
    adjacency = scipy.sparse.coo_matrix((np.ones(data.edge_index.shape[1]),
                      (data.edge_index[0, :], data.edge_index[1, :])),
                      shape=(num_nodes, num_nodes), dtype=np.float32)

    # Add the identity matrix to the adjacency matrix A
    A_hat = adjacency + scipy.sparse.eye(num_nodes)

    # Obtain D_hat
    degree = A_hat.sum(axis=0).A1
    D_hat = scipy.sparse.diags([degree], [0], dtype=np.float32)

    # Compute the inverse square root of the degree array
    degree_sqrt_inv = 1 / np.sqrt(degree)

    # Construct the matrix D^(-1/2) as a sparse diagonal matrix
    D_final = scipy.sparse.diags([degree_sqrt_inv], [0], dtype=np.float32)

    # normalize
    A_hat_norm = D_final.dot(A_hat).dot(D_final)
    return A_hat_norm

train_a_norm = [get_a_norm(data) for data in train_dataset]
valid_a_norm = [get_a_norm(data) for data in valid_dataset]
test_a_norm  = [get_a_norm(data) for data in test_dataset]
print(test_a_norm[0]) # For visualization

  (0, 0)	0.25
  (0, 1)	0.20412415266036987
  (0, 17)	0.25
  (0, 20)	0.22360679507255554
  (1, 0)	0.20412415266036987
  (1, 1)	0.16666667879725594
  (1, 2)	0.20412415266036987
  (1, 3)	0.20412415266036987
  (1, 17)	0.20412415266036987
  (1, 20)	0.18257419029314548
  (2, 1)	0.20412415266036987
  (2, 2)	0.25
  (2, 3)	0.25
  (2, 20)	0.22360679507255554
  (3, 1)	0.20412415266036987
  (3, 2)	0.25
  (3, 3)	0.25
  (3, 4)	0.25
  (4, 3)	0.25
  (4, 4)	0.25
  (4, 5)	0.25
  (4, 6)	0.22360679507255554
  (5, 4)	0.25
  (5, 5)	0.25
  (5, 6)	0.22360679507255554
  :	:
  (16, 18)	0.20412415266036987
  (16, 21)	0.25
  (17, 0)	0.25
  (17, 1)	0.20412415266036987
  (17, 17)	0.25
  (17, 20)	0.22360679507255554
  (18, 14)	0.18257419029314548
  (18, 15)	0.16666667879725594
  (18, 16)	0.20412415266036987
  (18, 18)	0.16666667879725594
  (18, 19)	0.20412415266036987
  (18, 21)	0.20412415266036987
  (19, 14)	0.22360679507255554
  (19, 15)	0.20412415266036987
  (19, 18)	0.20412415266036987
  (19, 19)	0.25
  (20, 0)	

#### Question 14 (1 pt)

To prepare the batch of graphs, we need to collect multiple adjacency matrices, feature matrices, and labels.

When using the trick described in the last figure, we see that we have to keep track of when a graph starts and when it ends, so that we can later differentiate the outputs due to $X^{(0)}$, $X^{(1)}$, etc. To achieve this, we will additionally collect a 1D array of batch indices, one for each $X^{(i)}$.

The 1D array has as many elements as rows in $X^{(i)}$, and it is filled with the value $i$ (the position of $X^{(i)}$ in the batch):

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/03-batch-indices.png">

We will later concatenate all the 1D arrays along the first dimension, just as we will do with all the $X^{(i)}$.

Define a function `prepare_graphs_batch()` that takes as input a dataset (e.g. `train_dataset`), and does the following

- Create four empty lists:
  - `adj_matrices`
  - `feature_matrices`
  - `batch_indices`
  - `labels`
- Iterate over the input dataset, getting one graph at a time. At each step, use your function from Question 13 to append the adjacency matrix to `adj_matrices`, append the matrix of input features to `feature_matrices`, create the array of batch indices (as explained above) and append it to `batch_indices`, and append the label of the graph to `labels`. **Make sure to convert the label to float**.
- Once the loop is over, use `scipy.sparse.block_diag()` to build the block diagonal matrix $\hat{A}_B$. Convert it to the COO format, and then use your answer to Question 6 to turn it into a sparse PyTorch tensor.
- Use `torch.cat()` to concatenate the tensors in `feature_matrices` along the first dimension. Do this also for `batch_indices` and `labels`.
- Return the 4 tensors computed in the previous two items.

In [15]:
def prepare_graphs_batch(dataset):
    # The four empty lists
    adj_matrices = []
    feature_matrices = []
    batch_indices = []
    labels = []
    
    for i, graph in enumerate(dataset):
        # Append the adjacency matrix
        adj_matrix = get_a_norm(graph)
        adj_matrices.append(adj_matrix)

        # Append the feature matrix 
        feature_matrices.append(graph.x)

        # Convert the batch index array to a tensor
        batch_index_tensor = torch.from_numpy(i * np.ones(graph.num_nodes, dtype=np.int64))
        batch_indices.append(batch_index_tensor)
        labels.append(graph.y)
    
    # Build block diagonal matrix in COO format and turn it into a sparse tensor
    a_norm_batch = scipy.sparse.block_diag(adj_matrices).tocoo()
    row_indices, col_indices, values = scipy.sparse.find(a_norm_batch)

    # Notice that it is here we we convert the labels into floats
    a_norm_batch_torch = torch.sparse_coo_tensor(torch.LongTensor([row_indices, col_indices]),
                                                 torch.FloatTensor(values),
                                                 a_norm_batch.shape)
    
    # Concatenate the feature, batch and label matrices using torch.cat
    features_batch = torch.cat(feature_matrices, dim=0)
    batch_indices_batch = torch.cat(batch_indices, dim=0)
    labels_batch = torch.cat(labels, dim=0)
    
    return a_norm_batch_torch, features_batch, batch_indices_batch, labels_batch

Once your answer for the previous question is ready, you can run the next cell to prepare all the required information, for the train, validation, and test splits.

In [16]:
train_a_norm, train_features, train_batch_idx, train_labels = prepare_graphs_batch(train_dataset)
valid_a_norm, valid_features, valid_batch_idx, valid_labels = prepare_graphs_batch(valid_dataset)
test_a_norm, test_features, test_batch_idx, test_labels = prepare_graphs_batch(test_dataset)

### GCNs for graph classification

We now have all the ingredients to pass a batch of graphs to a GCN. However, for each graph in the batch, the output $Z^{(i)}$ contains one row for each node in the graph. If the goal is to do classification at the graph level, we have to *pool* these vectors to then compute the required logits for classification.

This operation is similar as how pooling works in a CNN. We could consider taking the mean of the vectors, the sum, or use max-pooling. The difference with respect to CNNs is that in our case, we have a batch of graphs, each potentially with a different number of nodes.

To implement this specific pooling, we can use the scatter operation in the `torch_scatter` library, which comes when installing PyG. We will use it, together with the tensor of batch indices from the previous two questions, to pool the outputs of the GCN for each graph, into a single vector:

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/04-scatter.png">

You can check more details in the [documentation](https://pytorch-scatter.readthedocs.io/en/latest/functions/scatter.html).

#### Question 15 (1.0 pt)

Implement a `GraphClassifier` module using PyTorch.

- The constructor should take as arguments the number of input features, the hidden dimension, and the number of classes.
- The model should contain a instance of the `GCN` module (as you implemented it in Question 8). Use the same value for the hidden dimension and the number of output features (recall that your `GCN` module from Question 8 has two GCN layers).
- The model should also contain a `torch.nn.Linear` layer, with the hidden dimension as the input features, and the number of classes as the output.
- The forward method receives the concatenated matrix of features, the sparse block diagonal adjacency matrix, and the batch indices (the latter is used when calling `scatter`).
- Use the following architecture in the forward pass:
  - GCN $\to$ ReLU $\to$ scatter (max) $\to$ Linear.

The output of the forward should be a 1D tensor (you might need to call `squeeze` to get rid of extra dimensions) containing the logits for all graphs in the batch, for the binary classification task.

In [19]:
import torch_scatter

class GraphClassifier(nn.Module):
    def __init__(self, inp_feat, nhid, nclass):
        super(GraphClassifier, self).__init__()
        self.gcn = GCN(inp_feat, nhid, nclass)
        self.linear = nn.Linear(nhid, nclass)
    
    def forward(self, X, adj, batch_indices):
        out = self.gcn(X, adj)
        out = F.relu(out)
        out, _ = torch_scatter.scatter_max(out, batch_indices, dim=0)
        out = self.linear(out)
        return out.squeeze()

#### Question 16 (1.5 pt)

Implement a training loop for the graph classifier. Use the data from Question 14 to train and evaluate the model.

We encourage you to use a GPU in this section for faster training. Note that if you change the runtime at this point, you must re-execute several of the cells above, including the ones that install PyG.

- Instantiate a classifier with 32 as the hidden dimension
- Use Adam with a learning rate of 1e-3.
- Use `torch.nn.BCEWithLogitsLoss` as the loss function.
- Train for 5,000 epochs. Once training is done, plot the loss curve and the accuracy in the validation set. Then report the accuracy in the test set.

**Note:** the logits from the output of the classifier come from a linear layer. To compute actual predictions for the calculation of the accuracy, pass the logits through `torch.sigmoid()`, and set the predicted values to 1 whenever they are greater than 0.5, and to 0 otherwise.

You should get an accuracy equal to or higher than 70% in the validation and test sets. Can you beat the [state-of-the-art](https://paperswithcode.com/sota/graph-classification-on-proteins)? Feel free to modify your architecture and experiment with it.

Discuss what you observe during training and your results.

In [ ]:
# If your runtime is GPU-enabled, use .to(device) to move the model
# and all the relevant tensors to the GPU. You have to move tensors back to CPU
# when computing metrics like accuracy, using .cpu().
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

# Your answer here


## The end

If you have made it all the way here successfully, congratulations! 🎉 

You have implemented your own GCN and tested it on a node classification task, and a more challenging classification task over multiple graphs.

We hope you can use this knowledge to apply GCNs not only to the tasks described here, but other applications where data can be modeled as a graph.

If you are interested in applying graph neural networks to larger graphs, or try newer architectures, you can dive deeper into [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/), a library with fast implementations for a wide range of architectures. It also comes with custom code that takes care of aspects that you dealt with manually for this assignment, like a more efficient implementation of the adjacency matrix multiplication via message-passing methods, and Data Loaders that relieve you from having to build block diagonal sparse matrices.

You can also check the [Deep Graph Library](https://docs.dgl.ai/) another powerful library for deep learning on graphs which also integrates with other backends like TensorFlow.

# Grading (10pt)

- Question 1: 0.25pt 
- Question 2: 0.25pt 
- Question 3: 0.5pt 
- Question 4: 0.25pt 
- Question 5: 0.5pt 
- Question 6: 0.5pt 
- Question 7: 0.5pt 
- Question 8: 0.5pt 
- Question 9: 1.5pt 
- Question 10: 0.5pt 
- Question 11: 0.25pt 
- Question 12: 0.5pt 
- Question 13: 0.5pt 
- Question 14: 1pt
- Question 15: 1pt
- Question 16: 1.5pt